# Conexión con base de datos SQL e insercción de juegos

In [108]:
import mysql.connector
import pandas as pd 
from mysql.connector import errorcode
try:
    cnx = mysql.connector.connect(
      host="localhost",
      user="root",
      database='stum_for_you',
      passwd=""
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

# Corrección de fechas e ids en claves_dlc 

In [109]:
import datetime as dt
import random
cursor = cnx.cursor()
sql_select_query = """select * from claves_dlc"""
cursor.execute(sql_select_query)
a=cursor.fetchall()

for row in a:
    id_clave = row[0]
    if id_clave > 1000 :
        antigua = id_clave
        id_clave -= 100
        sql_update_query = """Update claves_dlc set id_clave = %s where id_clave = %s"""
        inputData = (id_clave,antigua)
        cursor.execute(sql_update_query,inputData)
    inicio = dt.datetime(2019, 1, 3)
    final =  dt.datetime(2020, 1, 18)
    fecha = inicio + (final - inicio) * random.random()
    
    sql_update_query = """Update claves_dlc set fecha_anexion = %s where id_clave = %s"""
    inputData = (fecha,id_clave)
    cursor.execute(sql_update_query,inputData)
cnx.commit()

# Insercción de juegos

In [5]:

cursor = cnx.cursor()
mySql_insert_query = """INSERT INTO juegos (clasificacion, descripcion, precio,nombre,genre_is_action, genre_is_adventure,
genre_is_casual, genre_is_strategy, genre_is_RPG, genre_is_simulation, genre_is_early_access, genre_is_free_to_play, 
genre_is_sports, genre_is_racing, genre_is_massively_multiplayer
) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """

try:
    data = pd.read_csv("games-features.csv")
    a=0
    for juego in data.iterrows() :
        a=a+1
        juego=juego[1]
        
        if (juego.GenreIsIndie) == True:
            clasificacion="INDIE"
        else:
            clasificacion="AAA"
        recordTuple = (clasificacion, juego.DetailedDescrip, juego.PriceInitial,juego.ResponseName,
                       juego.GenreIsAction, juego.GenreIsAdventure, juego.GenreIsCasual, juego.GenreIsStrategy,
                       juego.GenreIsRPG, juego.GenreIsSimulation, juego.GenreIsEarlyAccess, juego.GenreIsFreeToPlay,
                       juego.GenreIsSports, juego.GenreIsRacing, juego.GenreIsMassivelyMultiplayer)
        
        cursor.execute(mySql_insert_query, recordTuple)
        if(a==12699):
            break
        
    cnx.commit()
    cnx.close()
     
    print("Record inserted successfully into Laptop table")

except mysql.connector.Error as error:
    print("Failed to insert into MySQL table {}".format(error))
print(a)

E:\Users\Alberto\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Record inserted successfully into Laptop table
12699


# Corrección vencimiento fechas en la tabla descuentos

In [12]:
import datetime as dt
cursor = cnx.cursor()
sql_select_query = """select * from descuentos"""
cursor.execute(sql_select_query)
a=cursor.fetchall()

for row in a:

    id=row[0]
    if(id>800 and id <901):
        crea = dt.datetime(2018,12,1)
        fina = dt.datetime(2018,12,1)+dt.timedelta(weeks=+4)
        sql_update_query = """Update descuentos set fecha_creacion = %s where id_descuento = %s"""
        inputData = (crea,id)
        cursor.execute(sql_update_query,inputData)
        sql_update_query = """Update descuentos set fecha_vencimiento = %s where id_descuento = %s"""
        inputData = (fina,id)
        cursor.execute(sql_update_query,inputData)
    else:
        use_date = row[2]+dt.timedelta(weeks=+4)
        sql_update_query = """Update descuentos set fecha_vencimiento = %s where id_descuento = %s"""
        inputData = (use_date,id)
        cursor.execute(sql_update_query,inputData)
cnx.commit()
cnx.close()